# 🏥 Clinical Triage Router

**A specialized SLM agent (≤3B) for patient intake classification**

This notebook demonstrates:
1. **Synthetic Data Generation** - Gemini 1.5 Flash (free tier)
2. **Fine-Tuning** - Unsloth + LoRA on Qwen2.5-1.5B (4-bit)
3. **Agentic Loop** - Self-correction with retry logic
4. **Evaluation** - JSON validity & routing accuracy metrics

---
**Author**: Adarsh P  
**Runtime**: Google Colab T4 GPU (Free Tier)

## 🔧 Setup & Installation

In [ ]:
# Install dependencies (Colab T4 GPU)
!pip install -q unsloth
!pip install -q --no-deps xformers trl peft accelerate bitsandbytes
!pip install -q google-generativeai pydantic datasets

: 

In [ ]:
# Clone the repository
!git clone https://github.com/forkadarshp/clinic-triage.git
%cd clinic-triage

In [ ]:
# Verify imports
from src import config
from src.schemas import ToolName, parse_triage_output, get_mock_response

print(f"✅ Model: {config.MODEL_NAME}")
print(f"✅ Tools: {[t.value for t in ToolName]}")

---
## 📋 Phase 1: Synthetic Data Generation & Fine-Tuning

### 1.1 Generate Training Data (Gemini 1.5 Flash)

In [ ]:
import os
from google.colab import userdata

# Set API keys (Gemini or OpenAI supported)
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except: pass

try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except: pass

print("✅ API keys configured")

In [ ]:
from src.data_generator import generate_training_data, load_training_data

# Generate training examples
examples = generate_training_data(num_examples=config.NUM_TRAINING_EXAMPLES)
print(f"\n📊 Generated {len(examples)} examples")
print(f"\nSample:\n{examples[0]}")

### 1.2 Load Model (4-bit Quantized)

In [ ]:
from src.trainer import load_model_and_tokenizer, prepare_model_for_training

# Load pre-quantized model
model, tokenizer = load_model_and_tokenizer()
print(f"✅ Loaded {config.MODEL_NAME}")

# Add LoRA adapters
model = prepare_model_for_training(model)
print(f"✅ LoRA adapters added (r={config.LORA_R})")

### 1.3 Fine-Tune

In [ ]:
from src.trainer import prepare_dataset, train

# Prepare dataset
dataset = prepare_dataset(tokenizer)
print(f"📊 Training set: {len(dataset)} examples")

# Train
model = train(model, tokenizer, dataset)
print("\n✅ Training complete!")

---
## 🤖 Phase 2: The Agentic Loop

In [ ]:
from unsloth import FastLanguageModel

# Prepare model for inference
FastLanguageModel.for_inference(model)

In [ ]:
from src.agent import TriageAgent

# Initialize agent with trained model
agent = TriageAgent(model=model, tokenizer=tokenizer)

In [ ]:
# Test the agent
test_query = """65yo male, crushing chest pain radiating to jaw and left arm.
Started 20 minutes ago. Diaphoretic, nauseous. History of hypertension.
Location: 789 Pine Street."""

output, response, metadata = agent.run(test_query)

print("📋 Query:", test_query[:80], "...")
print("\n🔧 Tool Called:", output.tool if output else "FAILED")
print("📦 Arguments:", output.arguments.model_dump() if output else "N/A")
print("\n💬 Response:", response)
print(f"\n🔄 Attempts: {metadata['attempts']}")

---
## 📊 Phase 3: Evaluation

In [ ]:
from src.evaluator import load_test_data, evaluate, print_report

# Load 10 held-out test cases
test_data = load_test_data()
print(f"📋 Loaded {len(test_data)} test cases")

In [ ]:
# Run evaluation
results = evaluate(agent, test_data, verbose=True)

In [ ]:
# Print final report
print_report(results)

---
## 📝 Summary

| Metric | Result |
|--------|--------|
| **Model** | Qwen2.5-1.5B (4-bit) |
| **Training Examples** | config.NUM_TRAINING_EXAMPLES |
| **JSON Validity** | TBD% |
| **Routing Accuracy** | TBD% |
| **Routing MSE** | TBD |

### Key Features
- ✅ Runs on Colab Free Tier (T4 GPU)
- ✅ 4-bit quantization for memory efficiency
- ✅ Self-correction with 3-retry logic
- ✅ Pydantic validation for strict schema adherence
- ✅ Gemini Flash for zero-cost data generation